## Assignment-1
## Keerthi Bai reddy

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS bd23")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [5]:
# Created a Keyspace bd23
session.execute("CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':3}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [7]:
# Created a table "inventory" to store the inventory data
# The columns sku and warehouse_num are primary keys

session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory ( 
    sku INT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(sku,warehouse_num)
);
""")

### Import the Data

In [8]:
data = [
    (11,'laptop', 'HP Pavilion 8GB RAM 128GB SSD', 1),
    (12, 'Desk Chair', 'Comfortable ergonomic chair', 2),
    (13, 'Coffee Maker', 'Keurig Single Serve Coffee Maker', 123),
    (14, 'Running Shoes', 'Nike Air Zoom Pegasus', 123),
    (15, 'Backpack', 'Osprey Hiking Backpack', 10),
    (16, 'Cookware Set', 'Cuisinart Stainless Steel Cookware Set', 15)
]

for entry in data:
    insert_query = """ 
    INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
    VALUES (%s,%s,%s,%s)
    """

    session.execute(insert_query, entry)

In [9]:
# Query to get all the rows in inventory table

rows = session.execute("select (sku,name,description,warehouse_num) from bd23.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse={row[0][3]}")

sku=16, name=Cookware Set, description=Cuisinart Stainless Steel Cookware Set, warehouse=15
sku=13, name=Coffee Maker, description=Keurig Single Serve Coffee Maker, warehouse=123
sku=11, name=laptop, description=HP Pavilion 8GB RAM 128GB SSD, warehouse=1
sku=15, name=Backpack, description=Osprey Hiking Backpack, warehouse=10
sku=14, name=Running Shoes, description=Nike Air Zoom Pegasus, warehouse=123
sku=12, name=Desk Chair, description=Comfortable ergonomic chair, warehouse=2


In [10]:
# Query to get the rows with warehouse_num as 123 from the inventory table

rows = session.execute("select (sku,name,description,warehouse_num) from bd23.inventory where warehouse_num = 123 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

13, Coffee Maker, Keurig Single Serve Coffee Maker, 123
14, Running Shoes, Nike Air Zoom Pegasus, 123
